# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
import dask.dataframe as dd
from dotenv import load_dotenv

load_dotenv()


True

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [2]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*.parquet"), recursive=True)
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
dd_orig = dd.read_parquet(parquet_files).set_index("Ticker")
dd_orig.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year
Ticker,,,,,,,,
A,2000-01-03,43.382858,51.502148,56.464592,48.193848,56.330471,4674353.0,2000
A,2000-01-04,40.068871,47.567955,49.266811,46.316166,48.730328,4765083.0,2000
A,2000-01-05,37.583405,44.617310,47.567955,43.141991,47.389126,5758642.0,2000
A,2000-01-06,36.152374,42.918453,44.349072,41.577251,44.080830,2534434.0,2000
A,2000-01-07,39.165066,46.494991,47.165592,42.203148,42.247852,2819626.0,2000


In [9]:
dd_shift = dd_orig.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)
dd_shift2 = dd_shift.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

/var/folders/z0/jyftqgk93lj8m65xfl3gh9380000gn/T/ipykernel_36982/1822200947.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_orig.groupby('Ticker', group_keys=False).apply(
/var/folders/z0/jyftqgk93lj8m65xfl3gh9380000gn/T/ipykernel_36982/1822200947.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift2 = dd_shift.groupby('Ticker', group_keys=False).apply(


In [16]:
dd_return = dd_shift2.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)
dd_feat = dd_return.assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [17]:
df_feat = dd_feat.compute()
df_feat['ret_ma10'] = df_feat.groupby('Ticker')['Returns'].transform(lambda x: x.rolling(10).mean())


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

No We could calculate the moving avarage lazyly in dask which defers calculation until needed which would result in a more efficient system.
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.